In [2]:
#Import required libraries
import numpy as np
import pandas as pd
import sklearn
import requests
from sklearn.preprocessing import OneHotEncoder

#This is the api key
api = "2c74fd25dbbaf91b73daf87263d0b35f"

In [10]:
#gen_list is the list of all possible genres in database

x = requests.get('https://api.themoviedb.org/3/genre/movie/list?api_key=2c74fd25dbbaf91b73daf87263d0b35f&language=en-US')
dfTemp = pd.json_normalize(x.json()['genres'])

gen_list = dfTemp["name"].values.tolist()

In [44]:
#Collect 20 movies at a time from disvoer api
#This loop discovers 20 new movies according to a specific year
df = pd.DataFrame([])
IDs = pd.Series([],dtype = int)

for pageNum in range(1,6):
    
    #retrieve 20 movies
    x = requests.get('https://api.themoviedb.org/3/discover/movie?api_key=%s&language=en-US&sort_by=vote_count.desc&include_adult=false&include_video=false&page=%d&vote_count.gte=100&with_original_language=en'%(api,pageNum))
    dfTemp = pd.json_normalize(x.json()['results'])
    #store all 20 movie ids into SerTemp Series
    
    SerTemp = pd.Series(dfTemp["id"])
   
    #loop through each of the 20 movie ids one by one
    for index, ids in SerTemp.items():
        #retrieve movie detals
        y = requests.get('https://api.themoviedb.org/3/movie/%d?api_key=%s&language=en-US'%(ids,api))
        dfTemp = pd.json_normalize(y.json())
        #keep only relevant columns
        dfTemp = dfTemp[["title","release_date","budget","revenue","vote_count","vote_average","runtime","id","genres"]]
        
        #ge
        genrelist = []
        for dic in dfTemp["genres"].iloc[0]:
            genrelist.append(dic.get('name'))
        
            dfTemp.at[0,"genres"]=genrelist
    
    
            allitems = dfTemp["genres"].iloc[0]
            for gen in gen_list:
                colName = "genre_" + gen
                if gen in allitems:
                    dfTemp[colName] = 1
                else:
                    dfTemp[colName] = 0
            
        dfTemp.drop(["genres"],axis=1,inplace = True)
    
        df = pd.concat([df,dfTemp])
        
    

    adult                     backdrop_path          genre_ids      id  \
0   False  /s3TBrRGB1iav7gFOCNx3H31MoES.jpg      [28, 878, 12]   27205   
1   False  /rAiYTfKGqDCRIIqo664sY9XZIvQ.jpg      [12, 18, 878]  157336   
2   False  /pbEkjhdfP7yuDcMB78YEZwgD4IN.jpg   [18, 28, 80, 53]     155   
3   False  /9BBTo63ANSmhC4e6r62OJFuK2GL.jpg      [878, 28, 12]   24428   
4   False  /8I37NtDffNV7AZlDa7uDvvqhovU.jpg  [28, 12, 14, 878]   19995   
5   False  /en971MEXui9diirXlogOrPKmsEn.jpg       [28, 12, 35]  293660   
6   False  /mDfJG3LC3Dqb67AZ52x3Z0jU0uB.jpg      [12, 28, 878]  299536   
7   False  /hZkgoQYus5vegHoetLkCJzb17zJ.jpg       [18, 53, 35]     550   
8   False  /2UFxrUHVuSK3Tth7DvQQlF4mGTd.jpg      [28, 878, 12]  118340   
9   False  /suaEOtk1N1sgg2MTM7oZd2cfVp3.jpg           [53, 80]     680   
10  False  /3h1JZGDhZ8nzxdgvkxha0qBqi05.jpg    [35, 18, 10749]      13   
11  False  /5jkE2SzR5uR2egEb1rRhF22JyWN.jpg           [12, 14]     671   
12  False  /cyecB7godJ6kNHGONFjUyVN9OX

In [28]:
df

,title,release_date,budget,revenue,vote_count,vote_average,runtime,id,genre_Action,genre_Adventure,...,genre_History,genre_Horror,genre_Music,genre_Mystery,genre_Romance,genre_Science Fiction,genre_TV Movie,genre_Thriller,genre_War,genre_Western
0,Inception,2010-07-15,160000000,825532764,33060,8.361,148,27205,1,1,...,0,0,0,0,0,1,0,0,0,0
0,Interstellar,2014-11-05,165000000,701729206,30381,8.388,169,157336,0,1,...,0,0,0,0,0,1,0,0,0,0
0,The Dark Knight,2008-07-14,185000000,1004558444,29059,8.506,152,155,1,0,...,0,0,0,0,0,0,0,1,0,0
0,The Avengers,2012-04-25,220000000,1518815515,28106,7.707,143,24428,1,1,...,0,0,0,0,0,1,0,0,0,0
0,Avatar,2009-12-15,237000000,2920357254,28085,7.566,162,19995,1,1,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,The Green Mile,1999-12-10,60000000,286801374,14949,8.507,189,497,0,0,...,0,0,0,0,0,0,0,0,0,0
0,Frozen,2013-11-20,150000000,1274219009,14904,7.249,102,109445,0,1,...,0,0,0,0,0,0,0,0,0,0
0,La La Land,2016-11-29,30000000,447407695,14888,7.885,129,313369,0,0,...,0,0,1,0,1,0,0,0,0,0
0,Ratatouille,2007-06-28,150000000,623726000,14839,7.792,111,2062,0,0,...,0,0,0,0,0,0,0,0,0,0


In [50]:
#This loop discovers 20 new movies according to a specific year

def get_clean_data(year = 2010, api = api, vote_count_min = 100):    
    df = pd.DataFrame([])
    IDs = pd.Series([],dtype = int)
    
    #retrieve the total number of pages that have movies that fit 
    x = requests.get('https://api.themoviedb.org/3/discover/movie?api_key=%s&primary_release_year=%d&language=en-US&sort_by=vote_count.desc&include_adult=false&vote_count.gte=%d&include_video=false&page=%d&with_original_language=en'%(api,year,vote_count_min,1))
    totalPages = pd.json_normalize(x.json())["total_pages"].iloc[0]
    
    #print(totalPages)

    for pageNum in range(1,totalPages+1):
        #print(pageNum)
        #retrieve 20 movies
        x = requests.get('https://api.themoviedb.org/3/discover/movie?api_key=%s&primary_release_year=%d&language=en-US&sort_by=vote_count.desc&include_adult=false&include_video=false&page=%d&vote_count.gte=%d&with_original_language=en'%(api,year,pageNum,vote_count_min))
        dfTemp = pd.json_normalize(x.json()['results'])
        #store all 20 movie ids into SerTemp Series
        SerTemp = pd.Series(dfTemp["id"])

        #loop through each of the 20 movie ids one by one
        for index, ids in SerTemp.items():
            #retrieve movie detals
            y = requests.get('https://api.themoviedb.org/3/movie/%d?api_key=%s&language=en-US'%(ids,api))
            dfTemp = pd.json_normalize(y.json())
            #keep only relevant columns
            dfTemp = dfTemp[["title","release_date","budget","revenue","vote_count","vote_average","runtime","id","genres"]]

            #ge
            genrelist = []
            for dic in dfTemp["genres"].iloc[0]:
                genrelist.append(dic.get('name'))

                dfTemp.at[0,"genres"]=genrelist


                allitems = dfTemp["genres"].iloc[0]
                for gen in gen_list:
                    colName = "genre_" + gen
                    if gen in allitems:
                        dfTemp[colName] = 1
                    else:
                        dfTemp[colName] = 0

            dfTemp.drop(["genres"],axis=1,inplace = True)

            df = pd.concat([df,dfTemp])
    
    return df



In [55]:
df2 = get_clean_data(2009,vote_count_min=10)

In [56]:
dfnew = df